In [4]:
import numpy as np
import pandas as pd
import nltk
import re
data = pd.read_csv("movies_initial.csv") 
print(data.head())

   imdbID                                   title  year     rating runtime  \
0       1                              Carmencita  1894  NOT RATED   1 min   
1       5                        Blacksmith Scene  1893    UNRATED   1 min   
2       3                          Pauvre Pierrot  1892        NaN   4 min   
3       8  Edison Kinetoscopic Record of a Sneeze  1894        NaN   1 min   
4      10   Employees Leaving the Lumi�re Factory  1895        NaN   1 min   

                      genre    released              director writer  \
0        Documentary, Short         NaN  William K.L. Dickson    NaN   
1                     Short  1893-05-09  William K.L. Dickson    NaN   
2  Animation, Comedy, Short  1892-10-28         �mile Reynaud    NaN   
3        Documentary, Short  1894-01-09  William K.L. Dickson    NaN   
4        Documentary, Short  1895-03-22         Louis Lumi�re    NaN   

                       cast  ...  imdbRating  imdbVotes  \
0                Carmencita  ...       

In [5]:
movie_index = []
movie_id = []
movie_director = []
movie_cast=[]
movie_title = []
summary = []
genre = []

movie_id=data['imdbID']
movie_title=data['title']
movie_director=data['director']
movie_cast=data['cast']
genre=data['genre']
summary=data['fullplot']
language=data['language']

df = pd.DataFrame({ 'ID': movie_id, 'MovieTitle': movie_title, 'Director': movie_director,'Cast':movie_cast,'Genre': genre,'Language':language, 'Summary': summary})
df.drop(df[df['MovieTitle'].str.contains("�", case=False)].index, inplace = True)
df['ID1']=np.arange(1, len(df)+1)
df.index = np.arange(1, len(df)+1)


print(df.head(22))

     ID                                         MovieTitle  \
1     1                                         Carmencita   
2     5                                   Blacksmith Scene   
3     3                                     Pauvre Pierrot   
4     8             Edison Kinetoscopic Record of a Sneeze   
5    12                             The Arrival of a Train   
6    14                      Tables Turned on the Gardener   
7    29                                      Baby's Dinner   
8    23                                            The Sea   
9    91                             The House of the Devil   
10   75  The Conjuring of a Woman at the House of Rober...   
11  131                                   A Terrible Night   
12  132                               Une partie de cartes   
13  138                                  The Bewitched Inn   
14  192                             Ella Lola, a la Trilby   
15  211                                 A Trip to the Moon   
16  230 

In [6]:
import random
df.isna().sum()

df = df.drop(df[df['Genre'] == ''].index)
df = df.drop(df[df['Summary'] == ''].index)



In [7]:
def clean_summary(text):
    text = re.sub("\'", "", str(text))
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text

df['clean_summary'] = df['Summary'].apply(lambda x: clean_summary(x))
#df.head(2)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords


df['GenreString'] = df['Genre'].str.replace(',', '')

df["combined_text"] = df["clean_summary"] + " " + df["Cast"] + " " + df["GenreString"]
# print(len(df["combined_text"]))
df=df.head(2000)


tf = TfidfVectorizer(analyzer = "word", ngram_range=(1,2), min_df=0, stop_words='english')

#tfidf_matrix = tf.fit_transform(df['combined_text'])
tfidf_matrix = tf.fit_transform(df['combined_text'].values.astype('U'))

cosine =  cosine_similarity(tfidf_matrix, tfidf_matrix)

In [16]:
def get_title_from_index(Index):
    return df[df.ID1 == Index]["MovieTitle"]
def get_index_from_title(MovieTitle):
    return df[df.MovieTitle == MovieTitle]["ID1"].values[0]
listOfMovies=[]
def get_recommendations(movie):
    global listOfMovies
    movie_index = get_index_from_title(movie)
    similar_Movies = list(enumerate(cosine[movie_index]))
   
    sortedMovies = sorted(similar_Movies, key = lambda x:x[1], reverse=True)[1:]

    i = 0
    
    listOfMovies=[]
    
    for book in sortedMovies:
     
        

        listOfMovies.append(get_title_from_index(book[0]+1).to_string(index=False))
        i = i+1
        if i>=len(similar_Movies)-1:
            break
    listOfMovies =list(dict.fromkeys(listOfMovies))
    listOfMovies=listOfMovies[:25]


In [17]:
# t=input()
# get_recommendations(t)

In [ ]:

import tkinter as tk
from tkinter import *

mrs=Tk()
mrs.title("Movie Recommendation System")
mrs.geometry("710x650")

inp=""

C = Canvas(mrs, bg="blue", height=250, width=300)
filename = PhotoImage(file = "download.png")
background_label = Label(mrs, image=filename)
background_label.place(x=0, y=0, relwidth=1, relheight=1)


mrsLabel = Label(mrs,text="MOVIE RECOMMENDATION SYSTEM",bg='orange',font=("Arial",30,'italic'),relief=RAISED)
mrsLabel.pack(side=TOP)

Outputtxt=Text(mrs)
def printOutput():
    get_recommendations(inp)
    totCount=len(listOfMovies)
    Outputtxt.insert(tk.END,"RECOMMENDED MOVIE NAMES\n")
    for i in range(0,totCount):
        Outputtxt.insert(tk.END,listOfMovies[i]+"\n")
    Outputtxt.config(bg='white',bd=10,relief=RAISED)
    Outputtxt.pack()

def getInput():
        global inp
        inp = inputtxt.get(1.0, "end-1c")
        lbl.config(bg="light gray",text = "Provided Input: "+inp)
        printOutput()

def clrTxt():
    global inp
    inp=""
    inputtxt.delete("1.0","end")
    Outputtxt.delete("1.0","end")


lbl=Label(mrs,text="Film Name",font=("Arial",15,'italic'),relief=RAISED,bg="light gray")
lbl.pack(side=TOP,anchor='center')

inputtxt = Text(mrs, height = 2,width = 20,bg='white',relief=RAISED)
inputtxt.pack()

SubmitButton =Button(mrs,text = "Submit", command = getInput,bg="light gray",bd=5)
SubmitButton.place(x=200,y=200)
SubmitButton.pack()

ClearButton =Button(mrs,text = "Clear   ", command = clrTxt,bg="light gray",bd=5)
ClearButton.place(x=200,y=200)
ClearButton.pack()



   


mrs.configure(bg='light grey')
mrs.mainloop()
